# Building the design matrix (2)

In [29]:
%run /home/grg/git/alfa/nilearn-helper.py
%matplotlib inline
import pandas as pd
import json
matplotlib.rc('text', color='white')
matplotlib.rc('patch',facecolor='aliceblue', edgecolor='black', linewidth=1.0)

axondir = '/home/grg/data/ALFA_DWI'
spmdir = '/home/grg/spm/'

designmat_fp = osp.join(spmdir, 'designmatrix.xls')
subjects = json.load(open(osp.join(spmdir, 'data', 'subjects.json')))
print len(subjects)

514


In [7]:
def collect_images(spmdir, subjects, verbose=True):
    im_types = ['MD', 'FA', 'RD', 'Jacobians', 'L1', 'MD_pred', 'MD_corr']

    images = {}
    for s in subjects:
        for i, each in enumerate(im_types):
            item = glob(osp.join(spmdir, each, '*%s*.nii'%s))[0]
            images.setdefault(each, {}).update({s:item})
    return images

def getdict(data, key, column, column2=None, value=None):
    import string
    if not value is None:
        data = data[data[column2] == value]
        
    d1 = dict([(string.atoi(str(int(e))), v) for e, v in data[[key, column]].to_dict(orient='split')['data']])
    return d1

d = getdict(vvol, 'subject', 'Volume_mm3', 'StructName', 'Right-Lateral-Ventricle')
d

{10010: 7546.8,
 10013: 11640.2,
 10015: 9392.0,
 10016: 3088.8,
 10019: 9330.8,
 10023: 15559.3,
 10024: 3581.2,
 10025: 8858.3,
 10026: 15289.5,
 10028: 6893.6,
 10029: 9107.8,
 10032: 5573.1,
 10034: 6986.3,
 10035: 1981.8,
 10036: 19101.2,
 10038: 7537.1,
 10040: 3505.0,
 10041: 3063.6,
 10042: 3710.9,
 10049: 17083.7,
 10051: 5920.5,
 10052: 3782.5,
 10053: 7939.2,
 10056: 6674.4,
 10065: 6006.8,
 10070: 2375.7,
 10071: 2789.7,
 10081: 9847.7,
 10084: 29388.5,
 10090: 9292.1,
 10096: 4149.9,
 10099: 9184.5,
 10102: 12859.2,
 10106: 5327.7,
 10108: 5986.2,
 10113: 5829.4,
 10117: 5234.4,
 10118: 3461.8,
 10123: 10325.3,
 10134: 15315.7,
 10144: 6251.6,
 10150: 5050.8,
 10151: 6370.9,
 10156: 10731.0,
 10158: 6798.4,
 10160: 10108.0,
 10162: 19472.0,
 10166: 7437.7,
 10170: 16708.8,
 10178: 3860.3,
 10180: 4551.8,
 10182: 23702.4,
 10186: 4891.1,
 10199: 7542.3,
 10200: 12450.6,
 10212: 5722.9,
 10213: 5434.1,
 10217: 2565.8,
 10225: 10926.2,
 10226: 6671.1,
 10235: 3957.7,
 10239: 

In [36]:
# Reading main tables with metadata
demo_fp = osp.join(spmdir, 'data', 'Data_VBM_Alfa.csv')
subj_fp= osp.join(spmdir, 'data', 'subjects.json')
gend_fp= osp.join(spmdir, 'data', 'genders.json')
educ_fp = osp.join(spmdir, 'data', 'educyears.json')
tiv_fp = osp.join(spmdir, 'data', 'tiv.json')
fsvol_fp = osp.join(spmdir, 'data', 'aseg FS ALFA.xlsx')
ages_fp = osp.join(spmdir, 'data', 'age.json') # contains ages in days
jdg_fp = osp.join(spmdir, 'data', 'Ventricular volumes.xlsx')

demo = pd.read_csv(demo_fp)
subjects = json.load(open(subj_fp))
images = collect_images(spmdir=spmdir, subjects=subjects)
groups_size = [102, 44, 143, 160, 65]
groups_names = ['apoe23', 'apoe24', 'apoe33', 'apoe34', 'apoe44']
im_types = ['MD', 'FA', 'RD', 'Jacobians', 'L1', 'MD_pred', 'MD_corr']

# Sorting out groups
groups = []
start = 0
for size in groups_size:
    g = subjects[start:start + size]
    start = start + size
    groups.append(g)

ages = json.load(open(ages_fp))
genders = json.load(open(gend_fp))
educyears = json.load(open(educ_fp))
tivs = json.load(open(tiv_fp))
vvol = pd.read_excel(fsvol_fp)

# Reading ventricle volumes and TIV
tivs = getdict(demo, 'Subj_ID', 'Tot Intracranial Volume (mm^3)')

# FreeSurfer Volumes
vvol_l = getdict(vvol, 'subject', 'Volume_mm3', 'StructName', 'Left-Lateral-Ventricle')
vvol_r = getdict(vvol, 'subject', 'Volume_mm3', 'StructName', 'Right-Lateral-Ventricle')
vvol_li = getdict(vvol, 'subject', 'Volume_mm3', 'StructName','Left-Inf-Lat-Vent')
vvol_ri = getdict(vvol, 'subject', 'Volume_mm3', 'StructName','Right-Inf-Lat-Vent')
vvol_3 = getdict(vvol, 'subject', 'Volume_mm3', 'StructName','3rd-Ventricle')
vvol_4 = getdict(vvol, 'subject', 'Volume_mm3', 'StructName', '4th-Ventricle')

# Juando's Volumes
jdg = pd.read_excel(jdg_fp)
jdg_tivs = getdict(jdg, 'subject', 'Total Intracranial Volume')
jdg_vvol = getdict(jdg, 'subject', 'Ventricular Volume')
jdg_vvol = dict([(e,v*1000) for e,v in jdg_vvol.items()])
#vvol = dict([(str(e)[:5], v*1000.0 / tivs[str(e)[:5]]) for e, v in jdg[['subject', 'Ventricular Volume']].to_dict(orient='split')['data'] if str(e)[:5] in jdg_tivs])

In [78]:
data = []

for s in subjects:
    print s    
    im = [images[e][s] for e in im_types]
    row = [s]
    age = ages[str(s)]
    educyear = educyears[str(s)]
    group = [int(s in each) for each in groups]
    gender = genders[str(s)]
    fsvol_l = (vvol_l[s] + vvol_li[s]) / tivs[s]
    fsvol_r = (vvol_r[s] + vvol_ri[s]) / tivs[s]
    fsvol = fsvol_l + fsvol_r
    jdgvol = jdg_vvol[s] / tivs[s]
    
    
    # Adding the row to the table
    row.extend(im)
    row.extend(group)
    row.extend([age*e for e in group])
    row.extend([age*age*e for e in group])
    row.extend([gender, educyear, fsvol_l, fsvol_r, fsvol, jdgvol]) #, vvol_l[s], tivs[s]])
    data.append(row)
    #print row

10070
10102
10108
10235
10365
10419
10463
10530
10551
10563
10576
10630
10668
10692
10693
10703
10725
10737
10756
10809
11045
11047
11048
11133
11180
11225
11262
11305
11351
11360
11387
11407
11414
11416
11550
11610
11614
11638
11658
11691
11721
11737
11768
11798
11803
11858
11902
11941
11975
11979
12032
12067
12079
12121
12140
12174
12244
12252
12323
12324
12331
12399
12425
12445
12479
12484
12493
12516
12637
12659
12730
12783
12787
12841
12920
12970
12976
13008
13035
13075
13090
13118
13138
13235
21092
44046
44205
55166
55529
55538
55854
66089
66125
66159
66239
66240
66270
66312
77068
77093
77094
77195
10015
10041
10090
10150
10282
10486
10518
10536
10645
11038
11144
11191
11195
11252
11436
11597
12015
12122
12245
12417
12704
12772
12930
13054
13070
21002
21011
55351
55353
55355
55488
55539
55636
55793
66020
66094
66164
66341
66492
77109
77152
77191
77224
77241
10013
10023
10024
10028
10032
10036
10038
10040
10042
10049
10052
10053
10056
10081
10117
10118
10151
10156
10158
10166
1017

In [79]:
# Building general dataframe  
columns = ['subject', 'MD', 'FA', 'RD', 'Jacobians', 'L1', 'MD_pred', 'MD_corr', 
       'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 
       'age23', 'age24', 'age33', 'age34', 'age44', 
       'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 
       'gender', 'educyears', 'fsvol_l', 'fsvol_r', 'ventricles_FS', 'ventricles_JDG']
df = pd.DataFrame(data, columns=columns)
df.to_excel('/home/grg/spm/designmat/fullmatrix.xls')
df

,subject,MD,FA,RD,Jacobians,L1,MD_pred,MD_corr,Apoe2-3,Apoe2-4,...,agesq24,agesq33,agesq34,agesq44,gender,educyears,fsvol_l,fsvol_r,ventricles_FS,ventricles_JDG
0,10070,/home/grg/spm/MD/10070_MD_MNIspace_s.nii,/home/grg/spm/FA/10070_FA_MNIspace_s.nii,/home/grg/spm/RD/10070_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10070.nii,/home/grg/spm/L1/10070_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10070_MD_pred.nii,/home/grg/spm/MD_corr/10070_MD_corr.nii,1,0,...,0,0,0,0,0,10,2.597497,1.923379,4.520876,4.066362
1,10102,/home/grg/spm/MD/10102_MD_MNIspace_s.nii,/home/grg/spm/FA/10102_FA_MNIspace_s.nii,/home/grg/spm/RD/10102_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10102.nii,/home/grg/spm/L1/10102_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10102_MD_pred.nii,/home/grg/spm/MD_corr/10102_MD_corr.nii,1,0,...,0,0,0,0,1,18,8.516509,7.990710,16.507219,15.301898
2,10108,/home/grg/spm/MD/10108_MD_MNIspace_s.nii,/home/grg/spm/FA/10108_FA_MNIspace_s.nii,/home/grg/spm/RD/10108_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10108.nii,/home/grg/spm/L1/10108_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10108_MD_pred.nii,/home/grg/spm/MD_corr/10108_MD_corr.nii,1,0,...,0,0,0,0,0,15,6.327523,4.094092,10.421616,9.432586
3,10235,/home/grg/spm/MD/10235_MD_MNIspace_s.nii,/home/grg/spm/FA/10235_FA_MNIspace_s.nii,/home/grg/spm/RD/10235_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10235.nii,/home/grg/spm/L1/10235_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10235_MD_pred.nii,/home/grg/spm/MD_corr/10235_MD_corr.nii,1,0,...,0,0,0,0,0,8,4.549009,3.015192,7.564201,7.059961
4,10365,/home/grg/spm/MD/10365_MD_MNIspace_s.nii,/home/grg/spm/FA/10365_FA_MNIspace_s.nii,/home/grg/spm/RD/10365_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10365.nii,/home/grg/spm/L1/10365_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10365_MD_pred.nii,/home/grg/spm/MD_corr/10365_MD_corr.nii,1,0,...,0,0,0,0,0,12,3.788028,3.982449,7.770477,7.754194
5,10419,/home/grg/spm/MD/10419_MD_MNIspace_s.nii,/home/grg/spm/FA/10419_FA_MNIspace_s.nii,/home/grg/spm/RD/10419_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10419.nii,/home/grg/spm/L1/10419_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10419_MD_pred.nii,/home/grg/spm/MD_corr/10419_MD_corr.nii,1,0,...,0,0,0,0,0,8,1.907387,1.576661,3.484049,3.298463
6,10463,/home/grg/spm/MD/10463_MD_MNIspace_s.nii,/home/grg/spm/FA/10463_FA_MNIspace_s.nii,/home/grg/spm/RD/10463_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10463.nii,/home/grg/spm/L1/10463_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10463_MD_pred.nii,/home/grg/spm/MD_corr/10463_MD_corr.nii,1,0,...,0,0,0,0,0,18,4.324335,3.077469,7.401804,6.594490
7,10530,/home/grg/spm/MD/10530_MD_MNIspace_s.nii,/home/grg/spm/FA/10530_FA_MNIspace_s.nii,/home/grg/spm/RD/10530_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10530.nii,/home/grg/spm/L1/10530_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10530_MD_pred.nii,/home/grg/spm/MD_corr/10530_MD_corr.nii,1,0,...,0,0,0,0,0,15,4.226887,3.294929,7.521816,7.626067
8,10551,/home/grg/spm/MD/10551_MD_MNIspace_s.nii,/home/grg/spm/FA/10551_FA_MNIspace_s.nii,/home/grg/spm/RD/10551_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10551.nii,/home/grg/spm/L1/10551_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10551_MD_pred.nii,/home/grg/spm/MD_corr/10551_MD_corr.nii,1,0,...,0,0,0,0,1,12,9.033532,8.192711,17.226243,15.809334
9,10563,/home/grg/spm/MD/10563_MD_MNIspace_s.nii,/home/grg/spm/FA/10563_FA_MNIspace_s.nii,/home/grg/spm/RD/10563_RD_MNIspace_s.nii,/home/grg/spm/Jacobians/s6j_t1_10563.nii,/home/grg/spm/L1/10563_L1_MNIspace_s.nii,/home/grg/spm/MD_pred/10563_MD_pred.nii,/home/grg/spm/MD_corr/10563_MD_corr.nii,1,0,...,0,0,0,0,0,12,2.878528,3.566023,6.444551,6.276600


In [81]:
# Saving to Excel files
for each in im_types:
    print each
    columns1 = [each, 
       'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 
       'age23', 'age24', 'age33', 'age34', 'age44', 
       'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 
       'gender', 'educyears']
    df1 = pd.DataFrame(df, columns=columns1)
    dmat_fp = osp.join(spmdir, 'designmat', 'dm_%s_noventvol.xls'%each)
    df1.to_excel(dmat_fp, index=False)
    
    # With ventricles from FS
    columns1 = [each, 
       'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 
       'age23', 'age24', 'age33', 'age34', 'age44', 
       'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 
       'gender', 'educyears', 'ventricles_FS']
    df1 = pd.DataFrame(df, columns=columns1)
    dmat_fp = osp.join(spmdir, 'designmat', 'dm_%s_fsvol.xls'%each)
    df1.to_excel(dmat_fp, index=False)
    
    # With ventricles from JDG
    columns1 = [each, 
       'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 
       'age23', 'age24', 'age33', 'age34', 'age44', 
       'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 
       'gender', 'educyears', 'ventricles_JDG']
    df1 = pd.DataFrame(df, columns=columns1)
    dmat_fp = osp.join(spmdir, 'designmat', 'dm_%s_jdgvol.xls'%each)
    df1.to_excel(dmat_fp, index=False)

MD
FA
RD
Jacobians
L1
MD_pred
MD_corr


Analysis ready to be run.

In [9]:
import sys
sys.path.append('/home/grg/git/alfa/')

In [10]:
import multireg_spm12 as mreg

excel_fp = '/home/grg/spm/data/designmatrix2.xls' # table containing model data
paramtype = 'MD' # given column from the previous table
analysis_wd = '/tmp/dummy' # working directory
mask_fp = '/home/grg/spm/MNI_T1_brain_mask.nii' # explicit mask for the analysis
print 'Reading', excel_fp, 'looking up column', paramtype, '...'

a = mreg.multiple_regression_analysis(paramtype, excel_fp, analysis_wd, mask_fp, verbose=False)
#a.run()

Reading /home/grg/spm/data/designmatrix2.xls looking up column MD ...
